In [3]:
import sys
!{sys.executable} -m pip install openpyxl

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# ---------------------------
# Imports
# ---------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from collections import Counter
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk import ngrams
from textblob import TextBlob

In [25]:
# Complete Python EDA script for your Fake News dataset
# Replace DATA_PATH with your CSV file path if needed.
# DEV NOTE: The path below was provided in the conversation history; replace with actual CSV path.
DATA_PATH = "F:/PG-DBDA-2025/Project_Upload/FAKE-NEWS-CLASSIFIER/Data_using/bharatfakenewskosh.xlsx"  # <-- replace with "your_dataset.csv"

# ---------------------------
# Imports
# ---------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from collections import Counter
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk import ngrams
from textblob import TextBlob

# If nltk resources not downloaded, uncomment:
# nltk.download('stopwords')
# nltk.download('punkt')

# ---------------------------
# Utility functions
# ---------------------------
def clean_publish_date(text):
    """
    Normalize publish_date values like "9th July 2022" to a parseable date.
    Returns pd.Timestamp or NaT.
    """
    if pd.isna(text):
        return pd.NaT
    # Remove ordinal suffixes: 1st, 2nd, 3rd, 4th...
    text = str(text).strip()
    text = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', text, flags=re.IGNORECASE)
    # Try parsing common formats
    for fmt in ["%d %B %Y", "%d %b %Y", "%Y-%m-%d", "%d/%m/%Y"]:
        try:
            return pd.to_datetime(text, format=fmt)
        except Exception:
            continue
    # Fallback - try pandas parser
    try:
        return pd.to_datetime(text, dayfirst=True, errors='coerce')
    except Exception:
        return pd.NaT

def text_length_stats(series):
    """Return DataFrame with word count, char count stats for a text series."""
    s = series.fillna("")
    words = s.map(lambda x: len(str(x).split()))
    chars = s.map(lambda x: len(str(x)))
    return pd.Series({
        'count': s.shape[0],
        'non_null': s.map(bool).sum(),
        'mean_words': words.mean(),
        'median_words': words.median(),
        'std_words': words.std(),
        'mean_chars': chars.mean(),
    })

def top_n_words(series, n=30, language='english'):
    """Return top n words excluding stopwords."""
    text = " ".join(series.dropna().astype(str).tolist()).lower()
    tokens = nltk.word_tokenize(text)
    stops = set(stopwords.words(language))
    words = [w for w in tokens if w.isalpha() and w not in stops]
    return Counter(words).most_common(n)

def top_ngrams(series, ngram=2, top_n=20):
    """Return top n n-grams from a text series."""
    text = " ".join(series.dropna().astype(str).tolist()).lower()
    tokens = [t for t in nltk.word_tokenize(text) if t.isalpha()]
    ng = ngrams(tokens, ngram)
    return Counter(ng).most_common(top_n)

def simple_sentiment(text):
    """Return polarity & subjectivity using TextBlob (works on English)"""
    if pd.isna(text) or str(text).strip()=="":
        return pd.Series([np.nan, np.nan])
    tb = TextBlob(str(text))
    return pd.Series([tb.sentiment.polarity, tb.sentiment.subjectivity])

# ---------------------------
# Load dataset
# ---------------------------
print("Loading dataset:", DATA_PATH)
# attempt to load CSV; if path is image or wrong, user must replace with CSV.
try:
    df = pd.read_excel(DATA_PATH)
except Exception as e:
    print("Could not read as CSV. Please set DATA_PATH to your CSV file path. Error:", e)
    # create an empty df skeleton with columns from conversation for guidance
    cols = ["id","Author_Name","Fact_Check_Source","Source_Type","Statement","Eng_Trans_Statement",
            "News Body","Eng_Trans_News_Body","Media_Link","Publish_Date","Fact_Check_Link",
            "News_Category","Language","Region","Platform","Text","Video","Image","Label"]
    df = pd.DataFrame(columns=cols)
    print("Created empty skeleton DataFrame with expected columns. Replace DATA_PATH and re-run.")
    # Stop further execution to avoid misleading outputs
    raise SystemExit("Stop - set DATA_PATH to real CSV file and re-run.")

print("Shape:", df.shape)
display(df.head())

# ---------------------------
# PHASE 1: Basic overview
# ---------------------------
print("\n== Basic info ==")
print(df.info())

print("\n== Summary statistics (object cols) ==")
print(df.describe(include='object').T)

# Missing values per column
miss = df.isna().mean().sort_values(ascending=False)
print("\nMissing value fraction per column:\n", miss)

# Visual missing heatmap (save figure)
plt.figure(figsize=(12,4))
sns.heatmap(df.isna(), cbar=False)
plt.title("Missing data heatmap")
plt.tight_layout()
plt.savefig("missing_heatmap.png", dpi=150)
plt.close()

# Duplicates
dup_count = df.duplicated().sum()
print(f"\nDuplicate rows (full-row): {dup_count}")
# Check duplicates based on text fields (Statement + News Body)
dup_text = df.duplicated(subset=["Statement","News Body"]).sum() if set(["Statement","News Body"]).issubset(df.columns) else 0
print(f"Duplicate Statement+News Body: {dup_text}")

# ---------------------------
# PHASE 2: Data types & conversions
# ---------------------------
# Lower-case columns for convenience
df.columns = [c.strip() for c in df.columns]

# Convert Label to numeric if possible
if "Label" in df.columns:
    try:
        df["Label"] = pd.to_numeric(df["Label"], errors='coerce').astype('Int64')
    except Exception:
        pass

# Parse Publish_Date if present
if "Publish_Date" in df.columns:
    df["publish_date_parsed"] = df["Publish_Date"].apply(clean_publish_date)
    print("\nParsed publish_date sample:")
    print(df[["Publish_Date","publish_date_parsed"]].head())
    # Extract year/month/day
    df["pub_year"] = pd.DatetimeIndex(df["publish_date_parsed"]).year
    df["pub_month"] = pd.DatetimeIndex(df["publish_date_parsed"]).month
    df["pub_dayofweek"] = pd.DatetimeIndex(df["publish_date_parsed"]).day_name()
    # Plot monthly/yearly distribution
    plt.figure(figsize=(10,4))
    df["pub_month"].value_counts().sort_index().plot(kind='bar')
    plt.title("Count by publish month")
    plt.tight_layout()
    plt.savefig("publish_month_count.png", dpi=150)
    plt.close()

# ---------------------------
# PHASE 3: Categorical EDA
# ---------------------------
cat_cols = ["Author_Name","Fact_Check_Source","Source_Type","News_Category","Language","Region","Platform"]
cat_cols = [c for c in cat_cols if c in df.columns]

for c in cat_cols:
    print(f"\n-- Top values for {c} --")
    print(df[c].value_counts().head(15))
    # Save barplot for top 10
    plt.figure(figsize=(8,4))
    df[c].value_counts().head(10).plot(kind='barh')
    plt.title(f"Top 10 {c}")
    plt.tight_layout()
    plt.savefig(f"top_{c}.png", dpi=150)
    plt.close()

# Label distribution
if "Label" in df.columns:
    print("\nLabel distribution:")
    print(df["Label"].value_counts(dropna=False))
    plt.figure(figsize=(5,4))
    df["Label"].value_counts().plot(kind='bar')
    plt.title("Label distribution (False/True or 0/1)")
    plt.tight_layout()
    plt.savefig("label_distribution.png", dpi=150)
    plt.close()

# Media type counts (Text, Video, Image)
media_cols = [c for c in ["Text","Video","Image"] if c in df.columns]
if media_cols:
    print("\nMedia columns counts (non-null true/false):")
    for c in media_cols:
        # normalize common boolean-like values
        vc = df[c].fillna("no").astype(str).str.lower().value_counts()
        print(f"{c}:\n{vc}")
    # plot
    plt.figure(figsize=(6,4))
    df[media_cols].apply(lambda col: col.fillna("no").astype(str).str.lower().map(lambda x: 1 if x in ['yes','true','1','y'] else 0)).sum().plot(kind='bar')
    plt.title("Count of media types present")
    plt.tight_layout()
    plt.savefig("media_counts.png", dpi=150)
    plt.close()

# ---------------------------
# PHASE 4: Text EDA
# ---------------------------
# Choose English-translated columns when available for NLP parts
text_cols_candidates = ["Eng_Trans_Statement","Eng_Trans_News_Body","Statement","News Body"]
text_cols = [c for c in text_cols_candidates if c in df.columns]

print("\nText columns found:", text_cols)

# Basic length stats per text column
length_stats = {}
for c in text_cols:
    length_stats[c] = text_length_stats(df[c])
length_df = pd.DataFrame(length_stats).T
print("\nText length statistics:\n", length_df)
length_df.to_csv("text_length_stats.csv")

# Compare length by Label (if Label exists)
if "Label" in df.columns:
    for c in text_cols:
        plt.figure(figsize=(8,4))
        df_nonnull = df[[c,"Label"]].dropna(subset=[c])
        # create a words column
        df_nonnull["word_count"] = df_nonnull[c].astype(str).map(lambda x: len(x.split()))
        sns.boxplot(x="Label", y="word_count", data=df_nonnull)
        plt.title(f"Word count by Label for {c}")
        plt.tight_layout()
        plt.savefig(f"wordcount_by_label_{c}.png", dpi=150)
        plt.close()

# Top words and wordclouds
stop_eng = set(stopwords.words('english'))
for c in text_cols:
    # Top words
    top_words = top_n_words(df[c].astype(str).fillna(""), n=50, language='english')
    print(f"\nTop words for {c}:\n", top_words[:25])
    # WordCloud
    wc = WordCloud(width=800, height=400, collocations=False,
                   stopwords=stop_eng).generate(" ".join(df[c].dropna().astype(str).tolist()))
    plt.figure(figsize=(10,5))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"WordCloud: {c}")
    plt.tight_layout()
    plt.savefig(f"wordcloud_{c}.png", dpi=150)
    plt.close()
    # Top bigrams/trigrams
    print("Top bigrams:", top_ngrams(df[c].astype(str).fillna(""), ngram=2, top_n=20)[:10])
    print("Top trigrams:", top_ngrams(df[c].astype(str).fillna(""), ngram=3, top_n=10)[:8])

# ---------------------------
# PHASE 5: Sentiment analysis (on English-translated fields)
# ---------------------------
# We'll use Eng_Trans_News_Body if present else Eng_Trans_Statement
sent_col = None
for c in ["Eng_Trans_News_Body","Eng_Trans_Statement","News Body","Statement"]:
    if c in df.columns:
        sent_col = c
        break

if sent_col:
    print(f"\nPerforming simple sentiment analysis on {sent_col} using TextBlob (polarity/subjectivity).")
    df[[f"{sent_col}_polarity", f"{sent_col}_subjectivity"]] = df[sent_col].fillna("").astype(str).apply(simple_sentiment)
    # Save sentiment summary
    print(df[[f"{sent_col}_polarity", f"{sent_col}_subjectivity"]].describe())
    df[[f"{sent_col}_polarity", f"{sent_col}_subjectivity"]].to_csv("sentiment_summary.csv")
    # Compare polarity by label if available
    if "Label" in df.columns:
        plt.figure(figsize=(8,4))
        sns.boxplot(x="Label", y=f"{sent_col}_polarity", data=df.dropna(subset=[f"{sent_col}_polarity","Label"]))
        plt.title("Polarity by Label")
        plt.tight_layout()
        plt.savefig("polarity_by_label.png", dpi=150)
        plt.close()

# ---------------------------
# PHASE 6: Cross-analysis & correlations
# ---------------------------
# Example: Label vs News_Category
if "Label" in df.columns and "News_Category" in df.columns:
    ct = pd.crosstab(df["News_Category"], df["Label"], normalize='index')
    ct.to_csv("category_label_crosstab.csv")
    print("\nCategory vs Label crosstab (percent by category):\n", ct.head())

# Correlation between numeric-ish features: word counts, sentiment, etc.
numeric_feats = []
if sent_col:
    numeric_feats += [f"{sent_col}_polarity", f"{sent_col}_subjectivity"]
for c in text_cols:
    if c in df.columns:
        df[f"{c}_wordcount"] = df[c].fillna("").astype(str).map(lambda x: len(x.split()))
        numeric_feats.append(f"{c}_wordcount")

# Add binary media indicators
for m in media_cols:
    df[f"{m}_flag"] = df[m].fillna("no").astype(str).str.lower().map(lambda x: 1 if x in ['yes','true','1','y'] else 0)
    numeric_feats.append(f"{m}_flag")

# Label as numeric
if "Label" in df.columns:
    try:
        df["label_numeric"] = pd.to_numeric(df["Label"], errors='coerce')
        numeric_feats.append("label_numeric")
    except Exception:
        pass

if numeric_feats:
    corr = df[numeric_feats].corr()
    plt.figure(figsize=(8,6))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="vlag")
    plt.title("Correlation heatmap (numeric features)")
    plt.tight_layout()
    plt.savefig("numeric_correlation_heatmap.png", dpi=150)
    plt.close()
    corr.to_csv("numeric_feature_correlation.csv")
    print("\nSaved correlation matrix.")

# ---------------------------
# PHASE 7: Save cleaned summary outputs
# ---------------------------
# Basic cleaned snapshot (first N rows) and a CSV for EDA results
snapshot = df.head(500)
snapshot.to_csv("eda_snapshot_first500.csv", index=False)
print("\nSaved eda_snapshot_first500.csv")

# Summaries: missing, duplicates, top categories
summary = {
    "shape": df.shape,
    "missing_fraction": miss.to_dict(),
    "duplicates_rowcount": int(dup_count),
}
# Save summary as JSON-like CSV
pd.Series(summary).to_csv("eda_summary_overview.csv")
print("Saved eda_summary_overview.csv")

print("\nEDA complete. Figures saved as .png and summaries saved as CSVs in current working directory.")


Loading dataset: F:/PG-DBDA-2025/Project_Upload/FAKE-NEWS-CLASSIFIER/Data_using/bharatfakenewskosh.xlsx
Shape: (26232, 19)


,id,Author_Name,Fact_Check_Source,Source_Type,Statement,Eng_Trans_Statement,News Body,Eng_Trans_News_Body,Media_Link,Publish_Date,Fact_Check_Link,News_Category,Language,Region,Platform,Text,Video,Image,Label
0,BFNK_1,Shinjinee Majumder,Alt News,IFCN,फ़ैक्ट-चेक: तेलंगाना में एक रिपोर्टर ने गृह मंत...,Fact-check: A reporter in Telangana stopped sp...,सोशल मीडिया पर एक वीडियो वायरल है जिसमें एक पत...,A video is viral on social media in which a jo...,https://i0.wp.com/www.altnews.in/Hindi/wp-cont...,9th July 2022,https://www.altnews.in/Hindi/fact-check-was-am...,Politics,Hindi,Telangana,Twitter,no,yes,no,False
1,BFNK_2,Kalim Ahmed,Alt News,IFCN,PM मोदी को UAE का सर्वोच्च नागरिक सम्मान मिलने...,Share by stating the old video of PM Modi's hi...,प्रधानमंत्री नरेंद्र मोदी को सोने की चेन से सम...,A video of Prime Minister Narendra Modi being ...,https://i0.wp.com/www.altnews.in/Hindi/wp-cont...,9th July 2022,https://www.altnews.in/Hindi/old-video-of-pm-m...,Politics,Hindi,National,Twitter,no,yes,no,False
2,BFNK_3,Abhishek Kumar,Alt News,IFCN,वायरल तस्वीर में सुप्रीम कोर्ट के जज सूर्यकांत...,Supreme Court judges Suryakant and JB Pardiwal...,बीते दिनों नूपुर शर्मा ने टीवी डिबेट में पैगम्...,"Recently, Nupur Sharma made an objectionable c...",https://i0.wp.com/www.altnews.in/Hindi/wp-cont...,7th July 2022,https://www.altnews.in/Hindi/false-claim-with-...,Politics,Hindi,National,Twitter,no,no,yes,False
3,BFNK_4,Abhishek Kumar,Alt News,IFCN,मीडिया ने दी ग़लत ख़बर: कटनी में मुस्लिम सरपंच क...,Media gave wrong news: After the victory of Mu...,एक वीडियो सोशल मीडिया पर वायरल है. इसे शेयर कर...,A video is viral on social media. While sharin...,https://i0.wp.com/www.altnews.in/Hindi/wp-cont...,5th July 2022,https://www.altnews.in/Hindi/media-misreport-p...,Politics,Hindi,Madhya Pradesh,Twitter,no,no,yes,False
4,BFNK_5,Kinjal,Alt News,IFCN,महिला ने राहुल गांधी को कश्मीर मुद्दे पर मोदी ...,The woman lashed out at Rahul Gandhi to oppose...,सोशल मीडिया पर राहुल गांधी का एक वीडियो वायरल ...,A video of Rahul Gandhi has gone viral on soci...,https://i0.wp.com/www.altnews.in/Hindi/wp-cont...,4th July 2022,https://www.altnews.in/Hindi/2019-video-of-wom...,Politics,Hindi,Kashmir,Twitter,no,yes,no,True



== Basic info ==
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26232 entries, 0 to 26231
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   26232 non-null  object
 1   Author_Name          26005 non-null  object
 2   Fact_Check_Source    26232 non-null  object
 3   Source_Type          26232 non-null  object
 4   Statement            26232 non-null  object
 5   Eng_Trans_Statement  26232 non-null  object
 6   News Body            25359 non-null  object
 7   Eng_Trans_News_Body  25360 non-null  object
 8   Media_Link           24489 non-null  object
 9   Publish_Date         25417 non-null  object
 10  Fact_Check_Link      24302 non-null  object
 11  News_Category        26216 non-null  object
 12  Language             26232 non-null  object
 13  Region               25525 non-null  object
 14  Platform             26232 non-null  object
 15  Text                 26232 non-null

C:\Users\HP\AppData\Local\Temp\ipykernel_14852\3967137968.py:47: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(text, dayfirst=True, errors='coerce')



Parsed publish_date sample:
    Publish_Date publish_date_parsed
0  9th July 2022          2022-07-09
1  9th July 2022          2022-07-09
2  7th July 2022          2022-07-07
3  5th July 2022          2022-07-05
4  4th July 2022          2022-07-04

-- Top values for Author_Name --
Author_Name
admin               2005
Chendur Pandian     1666
Yogesh Karia        1320
Pankaj Iyer         1144
Vasuki S            1116
Mukundan K           920
Vikas Vyas           846
Siddharth Sahu       664
Saritadevi Samal     648
Dewin Carlos         639
Pooja Chaudhuri      622
OpIndia Staff -      579
Chayan Kundu         550
Kinjal               491
Archit Mehta         458
Name: count, dtype: int64

-- Top values for Fact_Check_Source --
Fact_Check_Source
Factcrescendo.com     10903
Alt News               3343
India Today            2495
Boomlive.in            2272
Youturn.in             1903
dfrac.org               903
OpIndia                 650
IndiaSpend              524
factchecker.in      

In [6]:
df.to_excel("cleaned_full_dataset.xlsx", index=False)


NameError: name 'df' is not defined